In [1]:
SEED = 91
import os
# to get reproducible results
import tensorflow as tf
tf.random.set_seed(SEED)
import numpy as np
np.random.seed(SEED)
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 

import pandas as pd
import kerastuner as kt # for hypermodel
from tensorflow import keras

from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
ohe = OneHotEncoder()
X = df.iloc[:, :-1].values

y = df.iloc[:, -1:].values
y_category = df.iloc[:, -1].astype('category').cat.codes
y_ohe = ohe.fit_transform(y).toarray()

input_size = X.shape[1]
size_output_layer = len(np.unique(y))

callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss" # Stop training when `val_loss` is no longer improving
                                               , min_delta=1e-5   # "no longer improving" being defined as "no better than 1e-5 less"
                                               , patience=3       # "no longer improving" being further defined as "for at least 3 epochs"
                                               , verbose=1 ) ]

In [4]:
class tuning(kt.HyperModel):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y

        self.input_size = X.shape[1]
        self.size_output_layer = len(set(y.reshape(1,-1)[0]))            
        
    def build(self, hp):

        keras.backend.clear_session()
        init = keras.initializers.GlorotUniform(seed=SEED)
        
        strategy = tf.distribute.MirroredStrategy()
        with strategy.scope():
            
            model = keras.models.Sequential()
            model.add(keras.layers.Dense(units = 100
                                         , input_dim=self.input_size
                                         , kernel_initializer=init
                                         , bias_initializer='zeros'
                                         , activation='tanh'
                                         , name='layer1'))


            model.add(keras.layers.Dense(self.size_output_layer, activation='softmax', name='layer3'))
        
        print(self.size_output_layer)
        hp_learning_rate = hp.Choice('learning_rate', values = [0.1] )
        hp_momentum = hp.Choice('momentum', values=[0.9] )
        hp_decay = hp.Choice('decay', values=[1e-6] )

        optimizer = keras.optimizers.SGD(learning_rate = hp_learning_rate
                                         , momentum=hp_momentum
                                         , decay=hp_decay
                                         , nesterov=True)
        
        model.compile(optimizer = optimizer
                      , loss = 'categorical_crossentropy'
                      , metrics = ['accuracy'])
        return model


In [5]:
keras.backend.clear_session()
init = keras.initializers.GlorotUniform(seed=SEED)

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units = 100
                                 , input_dim=input_size
                                 , kernel_initializer=init
                                 , bias_initializer='zeros'
                                 , activation='tanh'
                                 , name='layer1'))

    model.add(keras.layers.Dense(size_output_layer, activation='softmax', name='layer3'))

    optimizer = keras.optimizers.SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True) # the parameter from paper 

    model.compile(optimizer=optimizer
                      , loss='categorical_crossentropy'
                      , metrics=['accuracy'] )

    model.fit(X, y_ohe
              , epochs=100
              , batch_size=10
              , verbose=2
              , callbacks=callbacks
              , validation_split=0.1)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/100
28/28 - 1s - loss: 3.7621 - accuracy: 0.5441 - val_loss: 5.6746e-05 - val_accuracy: 1.0000
Epoch 2/100
28/28 - 0s - loss: 4.5110 - accuracy: 0.5257 - val_loss: 0.7395 - val_accuracy: 0.0000e+00
Epoch 3/100
28/28 - 0s - loss: 3.5082 - accuracy: 0.5588 - val_loss: 1.1921e-07 - val_accuracy: 1.0000
Epoch 4/100
28/28 - 0s - loss: 5.6057 - accuracy: 0.5257 - val_loss: 0.0165 - val_accuracy: 1.0000
Epoch 5/100
28/28 - 0s - loss: 3.7670 - accuracy: 0.5331 - val_loss: 4.7667 - val_accuracy: 0.0000e+00
Epoch 6/100
28/28 - 0s - loss: 4.7076 - accuracy: 0.4890 - val_loss: 21.8372 - val_accuracy: 0.0000e+00
Epoch 00006: early stopping


In [6]:
model_t = tuning(X, y_ohe)

tuner = kt.Hyperband(model_t
                     , objective = 'val_accuracy'
                     , max_epochs = 100
                     , overwrite = False
                     , directory = 'kt_dir'
                     , project_name = 'hp_tuning'
                     , seed = SEED
                     , executions_per_trial = 1
                    )

tuner.search(X , y_ohe
             , epochs=100
             , batch_size=10
             , verbose=2
             , validation_split=0.1)

INFO:tensorflow:Reloading Oracle from existing project kt_dir/hp_tuning/oracle.json
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
2
INFO:tensorflow:Reloading Tuner from kt_dir/hp_tuning/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [7]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0] 
print(best_hps.values)
keras.backend.clear_session()
model = tuner.hypermodel.build(best_hps)

{'learning_rate': 0.1, 'momentum': 0.9, 'decay': 1e-06, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
2


In [8]:
model.fit(X, y_ohe
              , epochs=100
              , batch_size=10
              , verbose=2
              , callbacks=callbacks
              , validation_split=0.1)

Epoch 1/100
28/28 - 0s - loss: 3.7957 - accuracy: 0.5404 - val_loss: 3.3541 - val_accuracy: 0.0000e+00
Epoch 2/100
28/28 - 0s - loss: 4.2591 - accuracy: 0.5551 - val_loss: 2.4743 - val_accuracy: 0.0000e+00
Epoch 3/100
28/28 - 0s - loss: 4.0988 - accuracy: 0.5404 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
28/28 - 0s - loss: 5.2307 - accuracy: 0.5404 - val_loss: 3.9392 - val_accuracy: 0.0000e+00
Epoch 5/100
28/28 - 0s - loss: 4.0308 - accuracy: 0.5037 - val_loss: 32.4639 - val_accuracy: 0.0000e+00
Epoch 6/100
28/28 - 0s - loss: 6.0510 - accuracy: 0.4816 - val_loss: 29.1292 - val_accuracy: 0.0000e+00
Epoch 00006: early stopping
